In [1]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import re

In [2]:
column_names = {'Emiasid': 'emiasid', 'Emiasid ': 'emiasid' , 'emiasid ': 'emiasid', 'EMIAS_ID': 'emiasid', 'МО': 'mo', 
                'МУ': 'mu', 'Логин оператора': 'login', 'Дата и время звонка': 'call_date', 'Результат звонка': 'call_result',
                'Результат звонка ': 'call_result', 'Разговор состоялся': 'talk_result', 'Комменатрий': 'comment', 
                'Дата и время переноса': 'next_call_date', 'Контактный телефон': 'phone_number', 'ФИО врача': 'doctor_fio', 
                'ФИО помощника врача': 'assistant_fio', 'Логин помощника врача': 'login', 'Зал №': 'hall_number',
                'Логин naumen': 'login', 'МО закрепления': 'mo', 'Отсутствует на рабочем месте': 'absence'}

In [3]:
today = pd.Timestamp(dt.datetime.today().date())
today_ymd = today.strftime('%Y%m%d')
today_y_m_d = today.strftime('%Y_%m_%d')

yesterday = today - dt.timedelta(days=1)
yesterday_ymd = yesterday.strftime('%Y%m%d')
yesterday_y_m_d = yesterday.strftime('%Y-%m-%d')

tomorrow = today + dt.timedelta(days=1)
tomorrow_ymd = tomorrow.strftime('%Y%m%d')
tomorrow_y_m_d = tomorrow.strftime('%Y-%m-%d')

print(today_y_m_d)

2022_09_16


In [4]:
# путь файла с текущими статусами пациентов
patients_current_statuses_path = \
r'\Users\murad_satabaev\Documents\GitHub\final_framework\merged_statuses'

patients_current_statuses_path = patients_current_statuses_path.split('\\')
patients_current_statuses_data = os.path.join('C:\\', r'\\', *patients_current_statuses_path,
                                             f'total_patients_current_statuses_merged_with_logins_{today_y_m_d}.xlsx')
print(patients_current_statuses_data)

C:\\Users\murad_satabaev\Documents\GitHub\final_framework\merged_statuses\total_patients_current_statuses_merged_with_logins_2022_09_16.xlsx


In [5]:
# путь файла с сегодняшними 
emias_data_path = r'\\t999\Сетевой диск\pdn\data\emias_data\xlsx'
emias_data_path = emias_data_path.split('\\')
emias_data_path = os.path.join(r'\\', *emias_data_path)
print(emias_data_path)

\\t999\Сетевой диск\pdn\data\emias_data\xlsx


In [6]:
today_emias_data_path = [os.path.join(emias_data_path, filename) for filename in os.listdir(emias_data_path) if '4059' 
                         in filename and '~' not in filename and today.strftime('%Y-%m-%d') in filename][0]
print(today_emias_data_path)

\\t999\Сетевой диск\pdn\data\emias_data\xlsx\DAT_4059_выгрузка_списков_на_обзвон_помощниками_2022-09-16.xlsx


In [7]:
# чтение всех файлов по отдельности, чтобы не тратить время на повторное чтение файла, который уже прочитан

In [8]:
today_emias_data_reading = pd.read_excel(today_emias_data_path)
today_emias_data_reading.shape[0]

97319

In [9]:
patients_current_statuses_reading = pd.read_excel(patients_current_statuses_data)
patients_current_statuses_reading.shape[0]

109888

In [27]:
import gspread
from gspread_dataframe import get_as_dataframe
gc = gspread.service_account(filename='google_sheets_credentials.json') 
url = 'https://docs.google.com/spreadsheets/d/1TXjbk9tRDMCjEyyiuK5OIgT8xwlHQ2V1PX2zaVnxPXY/edit#gid=0'
sh = gc.open_by_url(url)
ws = sh.get_worksheet(0)
def search_row(ws, desired_value):
    row_number = 0
    for row in ws.get_all_values():
        row_number += 1
        for value in row:
            if value == desired_value:
                return row_number
            
skiprows = search_row(ws, 'ФИО помощника врача') - 1

assistants_presence_reading = get_as_dataframe(ws, evaluate_formulas=True, skiprows=skiprows)
assistants_presence_reading = assistants_presence_reading.loc[~(assistants_presence_reading['Зал №'] == 'Зал №'), 
    ['Зал №', 'МО закрепления', 'ФИО помощника врача', 'Логин naumen', 'Отсутствует на рабочем месте']] \
        .dropna(how='all', axis=0)


print(dt.datetime.now())
print(assistants_presence_reading.shape[0])

assistants_presence = assistants_presence_reading.rename(columns=column_names).copy()

2022-09-16 09:13:07.738639
217


In [28]:
if assistants_presence[assistants_presence.absence.isna()].login.nunique() + \
assistants_presence[assistants_presence.absence == 1].login.nunique() == assistants_presence.login.nunique():
    print('\nВ сумме количество отсуствующих и присутствующих равны общему количеству помощников\n')
else:
    print(5 * '\nПРОВЕРЬТЕ ТАБЛИЦУ ОТСУТСТВУЮЩИХ И ПРИСУТСТВУЮЩИХ\n!!!\n')


В сумме количество отсуствующих и присутствующих равны общему количеству помощников



In [29]:
present_assistants = assistants_presence[assistants_presence.absence.isna()]
print(present_assistants.shape[0])
absent_assistants = assistants_presence[~assistants_presence.absence.isna()]
print(absent_assistants.shape[0])

164
53


In [30]:
patients_current_statuses = patients_current_statuses_reading.rename(columns=column_names).copy()

In [31]:
today_emias_data = today_emias_data_reading.rename(columns=column_names)

In [32]:
patients_current_statuses = patients_current_statuses_reading.rename(columns=column_names).copy()
patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')
patients_current_statuses['next_call_date'] = patients_current_statuses['next_call_date'] \
.apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
patients_current_statuses['last_call_date'] = patients_current_statuses['last_call_date'] \
.apply(lambda x: pd.to_datetime(x, dayfirst=True).date())

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_6132\1251941699.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')


In [ ]:
# Переведем все типы данных емиасайди, телефоны и прочие числовые данные в строки

In [33]:
today_emias_data['emiasid'] = today_emias_data['emiasid'].astype(str).str.replace('\.0', '')
today_emias_data['phone'] = today_emias_data['phone'].astype(str).str.replace('\.0', '')

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_6132\3971369908.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  today_emias_data['emiasid'] = today_emias_data['emiasid'].astype(str).str.replace('\.0', '')
C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_6132\3971369908.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  today_emias_data['phone'] = today_emias_data['phone'].astype(str).str.replace('\.0', '')


In [34]:
# путь папки с раскреплением
assistant_to_doctor_distribution_path = \
r'\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path'
assistant_to_doctor_distribution_path = \
assistant_to_doctor_distribution_path.split('\\')
assistant_to_doctor_distribution_path = \
os.path.join(r'\\', *assistant_to_doctor_distribution_path)
print(assistant_to_doctor_distribution_path)

\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path


In [35]:
september_assistant_to_doctor_distribution_path = [os.path.join(assistant_to_doctor_distribution_path, filename) for filename 
                                         in os.listdir(assistant_to_doctor_distribution_path) if '~' not in filename
                                        and 'september' in filename]
september_assistant_to_doctor_distribution_path.sort(key=lambda x: os.path.getctime(x))
september_assistant_to_doctor_distribution_path = september_assistant_to_doctor_distribution_path[-1]
print(september_assistant_to_doctor_distribution_path)

\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path\september_patient_to_assistant_distribution.xlsx


In [36]:
september_assistant_to_doctor_distribution = pd.read_excel(september_assistant_to_doctor_distribution_path, 
                                    usecols=['login', 'emiasid', 'mo', 'doctor_fio'])[['emiasid', 'login', 'mo', 'doctor_fio']]

In [37]:
doctors_list = set(september_assistant_to_doctor_distribution.doctor_fio.unique())
doctors_list.add(np.NaN)

In [38]:
def find_phone_number(text):
    """Функция ищет номера телефонов в строке, фильтрует дефектные и возвращает один с приоритетом на мобильный"""
    import re
    defect_pattern = re.compile(r"""7[^94]\d{9}|0{7,}|1{7,}|2{7,}|3{7,}|4{7,}|5{7,}|6{7,}|7{7,}|8{7,}|9{7,}""")
    text = str(text)
    text = ['7' + re.sub(r'\D', '', x)[-10:] for x in re.split(r'[,;]', text)]
    text = list(filter(lambda x: len(x) == 11, text))
    text = list(filter(lambda x: False if re.search(defect_pattern, x) else True, text))
    text = sorted(text, reverse=True)
    text = ','.join(text)
    text = re.search(r'(9\d{9}|4\d{9})', text)
    if text:
        text = text.group()
    return text

today_emias_data['phone_1'] = today_emias_data['phone'].apply(find_phone_number)
today_emias_data[today_emias_data['phone_1'].isna()].shape[0]

877

In [39]:
today_emias_data.shape[0]

97319

In [40]:
patients_current_statuses.shape[0]

109888

In [41]:
exception = pd.read_excel('exception.xlsx')

In [42]:
exception = exception.rename(columns=column_names)
exception['emiasid'] = exception['emiasid'].astype(str).str.replace('\.0', '')

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_6132\4054622638.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  exception['emiasid'] = exception['emiasid'].astype(str).str.replace('\.0', '')


In [279]:
exception.shape[0]

372

In [43]:
patients_current_statuses.problem.unique()

array([nan, 'Нет прикрепления к врачу', 'Нет в сегодняшней выгрузке',
       'Отсутствует номер', 'Нет прикрепления к МО', 'Смена МО',
       'Смена МО на другую нашу'], dtype=object)

In [156]:
actual_patients = \
patients_current_statuses[
    (~(patients_current_statuses.problem.isin({
    'Нет в сегодняшней выгрузке', 'Нет прикрепления к МО', 'Отсутствует номер', 'Смена МО', 'Смена МО на другую нашу'
    })))
    &
    (~(patients_current_statuses.emiasid.isin(exception.emiasid)))
]

actual_patients.shape[0]

107837

In [157]:
patients_current_statuses.shape[0]

109888

In [158]:
actual_patients.shape[0]

107837

In [159]:
actual_patients.status.value_counts()

Не дозвонились                    41232
Перенос звонка на другой день     30806
Будет в Москве в другом месяце    14408
Разговор состоялся                 6762
Ни разу не звонили                 5357
Запишется самостоятельно           4372
Перезвонить сегодня                1741
Пациент отрицает диагноз            867
Отказ                               676
Иная МО                             469
Госпитализирован                    354
Живет не в Москве                   282
Патронаж                            202
Некорректный номер                  114
Не нравится врач                     94
Другой врач в этой МО                32
Не нравится МО                       19
Cнят с ДН                            13
Отсутствует номер                    12
Умер                                  3
Name: status, dtype: int64

In [160]:
first_stage_patients = \
actual_patients[actual_patients.status == 'Ни разу не звонили']

first_stage_patients['priority'] = 2
first_stage_patients.shape[0]

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_6132\3421971818.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_stage_patients['priority'] = 2


5357

In [161]:
call_back_on_date_patients = \
actual_patients[
    (actual_patients.next_call_date.isin({
        dt.date(2022, 9, 14),
        dt.date(2022, 9, 15),
        dt.date(2022, 9, 16)
    }))
]

call_back_on_date_patients['priority'] = 1
call_back_on_date_patients.shape[0]

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_6132\1933833767.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  call_back_on_date_patients['priority'] = 1


5864

In [162]:
actual_patients[
    (actual_patients.month == 'august')
    &
    (actual_patients.last_call_date < dt.date(2022, 9, 2))
].shape[0]

4112

In [163]:
actual_patients.last_call_result.unique()

array(['Разговор состоялся', 'Нет ответа', 'Не дозвонились',
       'Перенос звонка на другой день', 'Перезвонить сегодня', 'Занято',
       'Отказ', 'Сброс', 'Информация о ДН', 'operator_rejected',
       'Госпитализирован', 'Номер не существует', nan], dtype=object)

In [164]:
august_patients = \
actual_patients[
    (actual_patients.month == 'august')
    &
    (actual_patients.last_call_result != 'Перенос звонка на другой день')
    &
    (actual_patients.last_call_date < dt.date(2022, 9, 2))
]

august_patients['priority'] = 3
august_patients.shape[0]

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_6132\1560472559.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  august_patients['priority'] = 3


2222

In [165]:
august_patients.last_call_date.unique()[-1]

datetime.date(2022, 9, 1)

In [166]:
tomorrow_record_patients_set = set()
for column in today_emias_data.columns[22:98:2]:
    tomorrow_record_patients_set = \
    tomorrow_record_patients_set | set(today_emias_data[today_emias_data[f'{column}'] == tomorrow_y_m_d]\
                                   .emiasid.astype(str).unique())

print(len(tomorrow_record_patients_set))

553


In [167]:
tomorrow_record_patients = patients_current_statuses[(patients_current_statuses.emiasid.isin(tomorrow_record_patients_set))
& (~(patients_current_statuses.status.isin({'Ни разу не звонили'}))) & 
(patients_current_statuses.last_call_result != 'Перенос звонка на другой день')]
print(tomorrow_record_patients.shape[0])

261


In [168]:
tomorrow_record_patients['priority'] = 4

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_6132\1903791372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tomorrow_record_patients['priority'] = 4


In [169]:
tomorrow_record_patients.status.value_counts()

Не дозвонились                    152
Разговор состоялся                 72
Будет в Москве в другом месяце     12
Запишется самостоятельно           12
Перезвонить сегодня                 5
Госпитализирован                    2
Пациент отрицает диагноз            2
Отказ                               1
Живет не в Москве                   1
Не нравится врач                    1
Другой врач в этой МО               1
Name: status, dtype: int64

In [170]:
tomorrow_record_patients['status'] = 'Напомнить'

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_6132\672541353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tomorrow_record_patients['status'] = 'Напомнить'


In [171]:
second_stage_patients = \
actual_patients[
    (actual_patients.status.isin({'Разговор состоялся', 'Не дозвонились', 'Другой врач в этой МО'
                                'Запишется самостоятельно', 'Не нравится МО', 'Не нравится врач'}))
    &
    (~(actual_patients.emiasid.isin(call_back_on_date_patients.emiasid)))
    &
    (~(actual_patients.last_call_date.isin(
        {
        dt.date(2022, 9, 13),
        dt.date(2022, 9, 14),
        dt.date(2022, 9, 15)
        }
    )))
]

second_stage_patients['priority'] = 5

second_stage_patients.shape[0]

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_6132\1793606296.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_stage_patients['priority'] = 5


27582

In [172]:
final_actual_patients_for_task_ = pd.concat([
    call_back_on_date_patients,
    first_stage_patients,
#     august_patients,
#     second_stage_patients,
#     tomorrow_record_patients
]) \
.sort_values(['priority']).reset_index(drop=True)

final_actual_patients_for_task_.shape[0]

11221

In [173]:
final_actual_patients_for_task = final_actual_patients_for_task_.drop_duplicates(subset=['phone_number'], keep='first')
final_actual_patients_for_task.shape[0]

11183

In [174]:
final_actual_patients_for_task = final_actual_patients_for_task.drop_duplicates(subset=['emiasid'], keep='first')
final_actual_patients_for_task.shape[0]

11183

In [175]:
final_actual_patients_for_task.head(3)

,emiasid,operator_login,status,old_comment,next_call_date,mo,doctor_fio,phone_number,calls_amount,last_call_date,last_call_result,last_talk_result,group,problem,month,operator_absence,priority
0,30547944,NaN,Будет в Москве в другом месяце,"перезвонить, 14 09 контр прием",2022-09-14,ГП 218,NaN,9.169243e+09,2.0,2022-09-13,Перенос звонка на другой день,NaN,21.0,NaN,august,NaN,1
1,2244575182,pomoshnik_pdn174,Перенос звонка на другой день,перезвонить,2022-09-16,ГП 22,Грошева Нина Александровна,9.629044e+09,1.0,2022-09-09,Перенос звонка на другой день,NaN,11.0,NaN,september,1.0,1
2,30000009674518,pomoshnik_pdn230,Перенос звонка на другой день,"перезвонить, была занята",2022-09-16,ГП 212,Милокост Инна Геннадиевна,9.039125e+09,1.0,2022-09-09,Перенос звонка на другой день,NaN,11.0,NaN,september,NaN,1


In [176]:
final_actual_patients_for_task = \
final_actual_patients_for_task[['emiasid', 'status', 'old_comment', 'operator_login', 'mo', 'doctor_fio', 'priority', 'calls_amount',
                                'last_call_date', 'month']]

In [177]:
final_actual_patients_for_task['emiasid'] = final_actual_patients_for_task['emiasid'].astype(str).str.replace('\.0', '')

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_6132\2998909243.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  final_actual_patients_for_task['emiasid'] = final_actual_patients_for_task['emiasid'].astype(str).str.replace('\.0', '')


In [178]:
initial_assistant_to_patient_distribution_in_today_task = \
final_actual_patients_for_task.copy()

In [179]:
initial_assistant_to_patient_distribution_in_today_task.old_comment.nunique()

2524

In [180]:
initial_assistant_to_patient_distribution_in_today_task.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(
    ['mo', 'emiasid']).head(20)

emiasid
mo     operator_login           
ГП 107 pomoshnik_pdn131        3
       pomoshnik_pdn130       20
       pomoshnik_pdn109       31
       pomoshnik_pdn43        31
       pomoshnik_pdn185       41
       pomoshnik_pdn129       45
       pomoshnik_pdn148       46
ГП 109 pomoshnik_pdn155       12
       pomoshnik_pdn134       35
       pomoshnik_pdn168       73
ГП 11  pomoshnik_pdn32        14
       pomoshnik_pdn184       18
       pomoshnik_pdn147       76
       pomoshnik_pdn146       83
ГП 134 pomoshnik_pdn7         33
       pomoshnik_pdn75        36
       pomoshnik_pdn89        37
       pomoshnik_pdn93        58
       pomoshnik_pdn63        65
ГП 166 pomoshnik_pdn209       44

In [181]:
set(present_assistants.login.unique()) - \
(set(initial_assistant_to_patient_distribution_in_today_task.operator_login.unique()) - {np.nan})

set()

In [182]:
# initial_assistant_to_patient_distribution_in_today_task.to_excel('initial_assistant_to_patient_distribution_in_today_task.xlsx')

In [183]:
task_amount = 300

In [184]:
%%time

initial_task_for_today = pd.DataFrame()
for operator_login in set(initial_assistant_to_patient_distribution_in_today_task.operator_login):
    initial_assistant_to_patient_distribution_in_today_task_for_operator_login = \
    initial_assistant_to_patient_distribution_in_today_task[initial_assistant_to_patient_distribution_in_today_task \
                                                            .operator_login
         == operator_login].sort_values(by=['priority', 'last_call_date', 'calls_amount'])
    call_back_number_for_operator_login = initial_assistant_to_patient_distribution_in_today_task_for_operator_login[
        initial_assistant_to_patient_distribution_in_today_task_for_operator_login.status.isin({
            'Перезвонить сегодня', 'Перенос звонка на другой день'})].shape[0]

    initial_task_for_today = pd.concat(
        [initial_task_for_today, 

         
         initial_assistant_to_patient_distribution_in_today_task[
             (initial_assistant_to_patient_distribution_in_today_task.operator_login==operator_login)
         ].head(task_amount),
        
       
        ]).reset_index(drop=True)

initial_task_for_today.shape[0]

CPU times: total: 1.28 s
Wall time: 1.28 s


10526

In [185]:
initial_task_for_today \
.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'])\
.sort_values(by='emiasid', ascending=True).head(10)

,,emiasid
mo,operator_login,
ГП 19,pomoshnik_pdn246,1
ГП 68,pomoshnik_pdn241,1
ГП 170,pomoshnik_pdn151,1
КДЦ 6,pomoshnik_pdn137,1
ГП 107,pomoshnik_pdn131,3
ГП 45,pomoshnik_pdn82,3
ГП 175,pomoshnik_pdn217,3
ДЦ 5,pomoshnik_pdn53,3
КДЦ 6,pomoshnik_pdn60,3


In [186]:
initial_task_for_today \
.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'])\
.sort_values(by='emiasid', ascending=False).head(10)

,,emiasid
mo,operator_login,
ГП 23,pomoshnik_pdn14,300
ГП 166,Помощник 7,300
ГП 214,pomoshnik_pdn156,300
ГП 2,pomoshnik_pdn83,300
ГП 191,pomoshnik_pdn13,300
ГП 23,pomoshnik_pdn10,300
ГП 166,Помощник 8,300
ГП 191,pomoshnik_pdn52,288
ГП 45,pomoshnik_pdn307,257


In [187]:
rename_mo = \
{'ГБУЗ Г МОСКВЫ ГП № 170 ДЗМ': 'ГП 170', ' ГП 6 ДЗМ': 'ГП 6', 'ГБУЗ КДЦ 6 ДЗМ':'КДЦ 6', 
'ГБУЗ "ГП № 134 ДЗМ"': 'ГП 134', 'КДЦ 4': 'КДЦ 4', 'ГБУЗ ДКЦ 1 ДЗМ': 'ДКЦ 1', 'ГБУЗ ГП 23 ДЗМ': 'ГП 23',
'ГБУЗ ГП 109 ДЗМ': 'ГП 109', 'ГБУЗ ГП 11 ДЗМ': 'ГП 11', 'ГБУЗ ГП 64 ДЗМ': 'ГП 64', 'ГБУЗ ГП 191 ДЗМ': 'ГП 191', 
'ГБУЗ ГП 220 ДЗМ': 'ГП 220', 'ГБУЗ "ГП № 218 ДЗМ"': 'ГП 218',  'ГБУЗ ГП 107 ДЗМ': 'ГП 107', 
'ГБУЗ "ГП № 22 ДЗМ"': 'ГП 22', 'ГБУЗ ДЦ 5 ДЗМ': 'ДЦ 5', 'ГБУЗ ГП 210 ДЗМ': 'ГП 210', 'ГБУЗ ГП 214 ДЗМ': 'ГП 214', 
'ГБУЗ ГП 2 ДЗМ': 'ГП 2', 'ГБУЗ КДП 121 ДЗМ': 'КДП 121', 'ГБУЗ ГП 66 ДЗМ': 'ГП 66',  'ГБУЗ ДЦ 3 ДЗМ ': 'ДЦ 3',
'ГБУЗ ГП 36 ДЗМ': 'ГП 36',  'ГБУЗ ГП 62 ДЗМ ': 'ГП 62', 
'ГБУЗ "ГП № 52 ДЗМ"': 'ГП 52', 'ГБУЗ "ГП 67 ДЗМ"': 'ГП 67', 'ГБУЗ ГП 68 ДЗМ': 'ГП 68', 'ГБУЗ ГП 175 ДЗМ': 'ГП 175',
'ГБУЗ ГП № 209 ДЗМ': 'ГП 209',  'ГБУЗ ГП 212 ДЗМ': 'ГП 212', 'ГБУЗ ГП 19 ДЗМ': 'ГП 19', 
'ГБУЗ ГП №166 ДЗМ': 'ГП 166', 'ГБУЗ ГП 9 ДЗМ': 'ГП 9', 'ГБУЗ ГП 69 ДЗМ': 'ГП 69',
'ГБУЗ "ГП № 3 ДЗМ"': 'ГП 3', 'ГБУЗ ГП 5 ДЗМ': 'ГП 5', 'ГБУЗ ГП 46 ДЗМ': 'ГП 46', 'ГБУЗ ГП №8 ДЗМ': 'ГП 8', 
'ГБУЗ "ГП № 45 ДЗМ"': 'ГП 45'}

In [188]:
mo_list = set(rename_mo.values())

In [189]:
new_doctor_patients = \
initial_assistant_to_patient_distribution_in_today_task[(initial_assistant_to_patient_distribution_in_today_task.mo.isin(mo_list)) & 
                    (~initial_assistant_to_patient_distribution_in_today_task.doctor_fio.isin(doctors_list))].reset_index(drop=True)

In [190]:
initial_task_for_today_present_assistants = \
initial_task_for_today[~initial_task_for_today.operator_login.isin(absent_assistants.login)]

In [191]:
initial_task_for_today[initial_task_for_today.operator_login.isin(absent_assistants.login)].shape[0]

3191

In [192]:
initial_task_for_today_present_assistants.shape[0]

7335

In [193]:
initial_task_for_today_present_assistants.groupby('operator_login').agg({'emiasid': 'count'}).sort_values('emiasid', ascending=False)

,emiasid
operator_login,
pomoshnik_pdn83,300
pomoshnik_pdn156,300
pomoshnik_pdn52,288
pomoshnik_pdn307,257
pomoshnik_pdn65,231
...,...
pomoshnik_pdn57,4
pomoshnik_pdn131,3
pomoshnik_pdn217,3


In [194]:
initial_task_for_today_absent_assistants =  \
initial_task_for_today[initial_task_for_today.operator_login.isin(absent_assistants.login)] \
[['emiasid', 'status', 'old_comment', 'mo', 'priority', 'month']]

In [195]:
additional_task_for_present_assistants = \
pd.concat([
#     new_doctor_patients,
    initial_task_for_today_absent_assistants
]).reset_index(drop=True)

In [196]:
additional_task_for_present_assistants_by_mo = \
additional_task_for_present_assistants.groupby(['mo']).agg({'emiasid': 'count'}).sort_values(by=['mo', 'emiasid'])

In [197]:
distribution_count = initial_task_for_today_present_assistants.groupby(['mo']) \
.agg({'emiasid': 'count', 'operator_login': 'nunique'}) \
.sort_values(by=['mo', 'emiasid'])

In [198]:
distribution_count['total_delta'] = distribution_count['operator_login'] * task_amount - distribution_count['emiasid']

In [199]:
distribution_count = distribution_count[distribution_count.total_delta > 0]

In [200]:
additional_task_for_present_assistants

,emiasid,status,old_comment,mo,priority,month
0,19732861,Перенос звонка на другой день,NaN,ДЦ 5,1,september
1,29568397,Перенос звонка на другой день,перезвонить,ДЦ 5,1,september
2,12665314668036,Перенос звонка на другой день,"нет слотов, актив 17745",ДЦ 5,1,september
3,19222665,Перенос звонка на другой день,перезвонить,ДЦ 5,1,september
4,30000007835971,Перенос звонка на другой день,перезвонить,ДЦ 5,1,september
...,...,...,...,...,...,...
3186,30000004609214,Будет в Москве в другом месяце,перезвонить,ДЦ 3,1,august
3187,20698427,Будет в Москве в другом месяце,Перезвонить,ДЦ 3,1,august
3188,30000009750609,Будет в Москве в другом месяце,Перезвонить,ДЦ 3,1,august
3189,20237995,Будет в Москве в другом месяце,Перезвонить,ДЦ 3,1,august


In [201]:
final_task_predistribution = \
pd.concat([initial_task_for_today_present_assistants, additional_task_for_present_assistants]).reset_index(drop=True)

In [202]:
empty_login_patients_amount_before = final_task_predistribution[final_task_predistribution.operator_login.isna()].shape[0]
empty_login_patients_amount_before

3191

In [203]:
%%time

for mo in distribution_count.index:
    
    while final_task_predistribution[final_task_predistribution.mo == mo].groupby('operator_login') \
    .agg({'emiasid': 'count'}).min()['emiasid'] < task_amount and \
    final_task_predistribution[(final_task_predistribution.operator_login.isna()) & (final_task_predistribution.mo == mo)] \
    .shape[0] > 0:
        
        miminal_in_mo_assistant_emiasid_amount = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('operator_login').agg({'emiasid': 'count'}).min()['emiasid']

        minimal_in_mo_emiasid_amount_login = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('operator_login').agg({'emiasid': 'count'}).sort_values(by='emiasid').index[0]
        
        
        empty_login_in_mo_index = final_task_predistribution[
            (final_task_predistribution.operator_login.isna()) & (final_task_predistribution.mo == mo)
        ].index[0]
        
        final_task_predistribution = \
        final_task_predistribution.sort_values(by=['priority', 'last_call_date', 'calls_amount'])

        final_task_predistribution.loc[empty_login_in_mo_index, 'operator_login'] = minimal_in_mo_emiasid_amount_login


CPU times: total: 43 s
Wall time: 43.2 s


In [204]:
empty_login_patients_amount_after = final_task_predistribution[final_task_predistribution.operator_login.isna()].shape[0]
empty_login_patients_amount_after

274

In [205]:
check_table = distribution_count.merge(additional_task_for_present_assistants_by_mo,left_index=True, right_index=True) \
[['total_delta', 'emiasid_y']]

In [206]:
check_table.loc[check_table['total_delta'] > check_table['emiasid_y'], 'real_addition'] = \
check_table.loc[check_table['total_delta'] > check_table['emiasid_y'], 'emiasid_y']

check_table.loc[check_table['total_delta'] <= check_table['emiasid_y'], 'real_addition'] = \
check_table.loc[check_table['total_delta'] <= check_table['emiasid_y'], 'total_delta']

In [207]:
real_change_of_emty_login_patients_amount = check_table.real_addition.sum()
real_change_of_emty_login_patients_amount

2917.0

In [208]:
empty_login_patients_amount_before - empty_login_patients_amount_after == real_change_of_emty_login_patients_amount

True

In [209]:
final_task_distribution = final_task_predistribution[~final_task_predistribution.operator_login.isna()] \
.copy().reset_index(drop=True)

In [210]:
final_task_distribution.shape[0]

10252

In [211]:
final_task_distribution.columns

Index(['emiasid', 'status', 'old_comment', 'operator_login', 'mo',
       'doctor_fio', 'priority', 'calls_amount', 'last_call_date', 'month'],
      dtype='object')

In [212]:
final_task_distribution.status.value_counts()

Перенос звонка на другой день     4744
Ни разу не звонили                4505
Перезвонить сегодня                371
Запишется самостоятельно           356
Будет в Москве в другом месяце     163
Не дозвонились                      90
Разговор состоялся                  11
Отказ                                6
Госпитализирован                     4
Пациент отрицает диагноз             1
Живет не в Москве                    1
Name: status, dtype: int64

In [213]:
final_task_distribution.shape[0]

10252

In [214]:
final_task_distribution.columns

Index(['emiasid', 'status', 'old_comment', 'operator_login', 'mo',
       'doctor_fio', 'priority', 'calls_amount', 'last_call_date', 'month'],
      dtype='object')

In [215]:
final_task_distribution.head(1)

,emiasid,status,old_comment,operator_login,mo,doctor_fio,priority,calls_amount,last_call_date,month
0,16911653,Перенос звонка на другой день,"перезвонить , записать к терапевту",pomoshnik_pdn168,ГП 109,Сафиуллина Валерия Романовна,1,2.0,2022-08-31,august


In [216]:
second_stage_patients.head(1)

,emiasid,operator_login,status,old_comment,next_call_date,mo,doctor_fio,phone_number,calls_amount,last_call_date,last_call_result,last_talk_result,group,problem,month,operator_absence,priority
5380,18937181,pomoshnik_pdn24,Разговор состоялся,NaN,NaT,КДЦ 4,Лосев Антон Сергеевич,9.852178e+09,2.0,2022-08-22,Разговор состоялся,NaN,13.0,NaN,august,NaN,5


In [217]:
second_stage_patients = second_stage_patients[['emiasid', 'status', 'old_comment', 'operator_login', 'mo', 'priority',
       'calls_amount', 'last_call_date', 'month']]

In [218]:
tomorrow_record_patients = tomorrow_record_patients[['emiasid', 'status', 'old_comment', 'operator_login', 'mo', 'priority',
       'calls_amount', 'last_call_date', 'month']]

In [219]:
tomorrow_record_patients

,emiasid,status,old_comment,operator_login,mo,priority,calls_amount,last_call_date,month
1938,19566964,Напомнить,NaN,pomoshnik_pdn66,ГП 22,4,6.0,2022-09-15,august
12895,15982363,Напомнить,NaN,pomoshnik_pdn77,ДКЦ 1,4,1.0,2022-09-02,september
13477,16773466,Напомнить,NaN,pomoshnik_pdn93,ГП 134,4,1.0,2022-09-02,september
14137,10320085,Напомнить,NaN,pomoshnik_pdn69,ГП 218,4,2.0,2022-09-13,september
14322,17539456,Напомнить,NaN,pomoshnik_pdn108,ГП 62,4,1.0,2022-09-02,september
...,...,...,...,...,...,...,...,...,...
108973,24638207,Напомнить,Запишется самостоятельно,pomoshnik_pdn66,ГП 22,4,2.0,2022-09-14,september
109123,25994509,Напомнить,Запишется самостоятельно,pomoshnik_pdn202,ГП 19,4,2.0,2022-09-14,september
109635,449927766,Напомнить,NaN,pomoshnik_pdn67,ГП 218,4,3.0,2022-09-15,september
109837,24207475,Напомнить,Кипова З. М.,pomoshnik_pdn109,ГП 107,4,1.0,2022-09-09,september


In [220]:
final_actual_patients_for_task_ = pd.concat([
    final_task_distribution,
    august_patients,
    second_stage_patients,
    tomorrow_record_patients
]) \
.sort_values(['priority']).reset_index(drop=True)

final_actual_patients_for_task_.shape[0]

40317

In [221]:
# final_actual_patients_for_task = final_actual_patients_for_task_.drop_duplicates(subset=['phone_number'], keep='first')
# final_actual_patients_for_task_.shape[0]

In [222]:
final_actual_patients_for_task_ = final_actual_patients_for_task_.drop_duplicates(subset=['emiasid'], keep='first')
final_actual_patients_for_task_.shape[0]

40156

In [223]:
final_actual_patients_for_task_.head(3)

,emiasid,status,old_comment,operator_login,mo,doctor_fio,priority,calls_amount,last_call_date,month,next_call_date,phone_number,last_call_result,last_talk_result,group,problem,operator_absence
0,16911653,Перенос звонка на другой день,"перезвонить , записать к терапевту",pomoshnik_pdn168,ГП 109,Сафиуллина Валерия Романовна,1,2.0,2022-08-31,august,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18577640,Перенос звонка на другой день,NaN,pomoshnik_pdn25,ГП 9,Ильина Татьяна Александровна,1,2.0,2022-09-13,september,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17897612,Перенос звонка на другой день,не хочет наблюдаться,pomoshnik_pdn217,ГП 175,Тихонов Константин Андреевич,1,2.0,2022-09-13,september,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [224]:
final_actual_patients_for_task_ = \
final_actual_patients_for_task_[['emiasid', 'status', 'old_comment', 'operator_login', 'mo', 'priority', 'calls_amount',
                                'last_call_date', 'month']]

In [225]:
final_actual_patients_for_task_['emiasid'] = final_actual_patients_for_task_['emiasid'].astype(str).str.replace('\.0', '')

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_6132\309388853.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  final_actual_patients_for_task_['emiasid'] = final_actual_patients_for_task_['emiasid'].astype(str).str.replace('\.0', '')


In [226]:
initial_assistant_to_patient_distribution_in_today_task = \
final_actual_patients_for_task_.copy()

In [227]:
initial_assistant_to_patient_distribution_in_today_task.old_comment.nunique()

3209

In [228]:
initial_assistant_to_patient_distribution_in_today_task.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(
    ['mo', 'emiasid']).head(20)

emiasid
mo     operator_login           
ГП 107 pomoshnik_pdn131        3
       pomoshnik_pdn130       61
       pomoshnik_pdn148      217
       pomoshnik_pdn43       256
       pomoshnik_pdn129      269
       pomoshnik_pdn185      290
       pomoshnik_pdn109      292
ГП 109 pomoshnik_pdn155       35
       pomoshnik_pdn134      201
       pomoshnik_pdn168      248
ГП 11  pomoshnik_pdn32        75
       pomoshnik_pdn146      207
       pomoshnik_pdn184      304
       pomoshnik_pdn147      364
ГП 134 pomoshnik_pdn63       242
       pomoshnik_pdn93       242
       pomoshnik_pdn75       244
       pomoshnik_pdn7        273
       pomoshnik_pdn89       352
ГП 166 pomoshnik_pdn209        1

In [229]:
# initial_assistant_to_patient_distribution_in_today_task.to_excel('initial_assistant_to_patient_distribution_in_today_task.xlsx')

In [230]:
set(present_assistants.login.unique()) - \
(set(initial_assistant_to_patient_distribution_in_today_task.operator_login.unique()) - {np.nan})

set()

In [231]:
task_amount = 120

In [232]:
%%time

initial_task_for_today = pd.DataFrame()
for operator_login in set(initial_assistant_to_patient_distribution_in_today_task.operator_login):
    initial_assistant_to_patient_distribution_in_today_task_for_operator_login = \
    initial_assistant_to_patient_distribution_in_today_task[initial_assistant_to_patient_distribution_in_today_task \
                                                            .operator_login
         == operator_login].sort_values(by=['priority', 'last_call_date', 'calls_amount'])
    call_back_number_for_operator_login = initial_assistant_to_patient_distribution_in_today_task_for_operator_login[
        initial_assistant_to_patient_distribution_in_today_task_for_operator_login.status.isin({
            'Перезвонить сегодня', 'Перенос звонка на другой день'})].shape[0]

    initial_task_for_today = pd.concat(
        [initial_task_for_today, 

         
         initial_assistant_to_patient_distribution_in_today_task[
             (initial_assistant_to_patient_distribution_in_today_task.operator_login==operator_login)
         ].head(task_amount),
        
       
        ]).reset_index(drop=True)

initial_task_for_today.shape[0]

CPU times: total: 2.64 s
Wall time: 2.63 s


22028

In [233]:
initial_task_for_today \
.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'])\
.sort_values(by='emiasid', ascending=True).head(10)

emiasid
mo     operator_login           
ГП 170 pomoshnik_pdn151        1
ГП 166 Помощник 7              1
       pomoshnik_pdn209        1
ГП 107 pomoshnik_pdn131        3
ДЦ 5   pomoshnik_pdn53         6
ДЦ 3   pomoshnik_pdn98        11
ГП 69  pomoshnik_pdn216       15
ГП 6   pomoshnik_pdn173       16
ГП 68  pomoshnik_pdn240       18
       pomoshnik_pdn242       20

In [234]:
initial_task_for_today \
.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'])\
.sort_values(by='emiasid', ascending=False).head(10)

emiasid
mo    operator_login           
ГП 45 pomoshnik_pdn292      120
ГП 66 pomoshnik_pdn249      120
ГП 64 pomoshnik_pdn45       120
      pomoshnik_pdn47       120
      pomoshnik_pdn79       120
      pomoshnik_pdn80       120
ГП 66 pomoshnik_pdn116      120
      pomoshnik_pdn159      120
      pomoshnik_pdn176      120
      pomoshnik_pdn21       120

In [235]:
rename_mo = \
{'ГБУЗ Г МОСКВЫ ГП № 170 ДЗМ': 'ГП 170', ' ГП 6 ДЗМ': 'ГП 6', 'ГБУЗ КДЦ 6 ДЗМ':'КДЦ 6', 
             'ГБУЗ "ГП № 134 ДЗМ"': 'ГП 134', 'КДЦ 4': 'КДЦ 4', 'ГБУЗ ДКЦ 1 ДЗМ': 'ДКЦ 1', 'ГБУЗ ГП 23 ДЗМ': 'ГП 23',
             'ГБУЗ ГП 109 ДЗМ': 'ГП 109', 'ГБУЗ ГП 11 ДЗМ': 'ГП 11', 'ГБУЗ ГП 64 ДЗМ': 'ГП 64', 'ГБУЗ ГП 191 ДЗМ': 'ГП 191', 
             'ГБУЗ ГП 220 ДЗМ': 'ГП 220', 'ГБУЗ "ГП № 218 ДЗМ"': 'ГП 218',  'ГБУЗ ГП 107 ДЗМ': 'ГП 107', 
             'ГБУЗ "ГП № 22 ДЗМ"': 'ГП 22', 'ГБУЗ ДЦ 5 ДЗМ': 'ДЦ 5', 'ГБУЗ ГП 210 ДЗМ': 'ГП 210', 'ГБУЗ ГП 214 ДЗМ': 'ГП 214', 
             'ГБУЗ ГП 2 ДЗМ': 'ГП 2', 'ГБУЗ КДП 121 ДЗМ': 'КДП 121', 'ГБУЗ ГП 66 ДЗМ': 'ГП 66',  'ГБУЗ ДЦ 3 ДЗМ ': 'ДЦ 3',
             'ГБУЗ ГП 36 ДЗМ': 'ГП 36',  'ГБУЗ ГП 62 ДЗМ ': 'ГП 62', 
            'ГБУЗ "ГП № 52 ДЗМ"': 'ГП 52', 'ГБУЗ "ГП 67 ДЗМ"': 'ГП 67', 'ГБУЗ ГП 68 ДЗМ': 'ГП 68', 'ГБУЗ ГП 175 ДЗМ': 'ГП 175',
'ГБУЗ ГП № 209 ДЗМ': 'ГП 209',  'ГБУЗ ГП 212 ДЗМ': 'ГП 212', 'ГБУЗ ГП 19 ДЗМ': 'ГП 19', 
'ГБУЗ ГП №166 ДЗМ': 'ГП 166', 'ГБУЗ ГП 9 ДЗМ': 'ГП 9', 'ГБУЗ ГП 69 ДЗМ': 'ГП 69',
'ГБУЗ "ГП № 3 ДЗМ"': 'ГП 3', 'ГБУЗ ГП 5 ДЗМ': 'ГП 5', 'ГБУЗ ГП 46 ДЗМ': 'ГП 46', 'ГБУЗ ГП №8 ДЗМ': 'ГП 8', 
'ГБУЗ "ГП № 45 ДЗМ"': 'ГП 45'}

In [236]:
mo_list = set(rename_mo.values())

In [237]:
# new_doctor_patients = \
# initial_assistant_to_patient_distribution_in_today_task[(initial_assistant_to_patient_distribution_in_today_task.mo.isin(mo_list)) & 
#                     (~initial_assistant_to_patient_distribution_in_today_task.doctor_fio.isin(doctors_list))].reset_index(drop=True)
# new_doctor_patients = new_doctor_patients[~new_doctor_patients.emiasid.isin(calls_data.emiasid)]

In [238]:
initial_task_for_today_present_assistants = \
initial_task_for_today[~initial_task_for_today.operator_login.isin(absent_assistants.login)]

In [239]:
initial_task_for_today[initial_task_for_today.operator_login.isin(absent_assistants.login)].shape[0]

4044

In [240]:
initial_task_for_today_present_assistants.shape[0]

17984

In [241]:
initial_task_for_today_present_assistants.groupby('operator_login').agg({'emiasid': 'count'}).sort_values('emiasid', ascending=False)

,emiasid
operator_login,
pomoshnik_pdn100,120
pomoshnik_pdn37,120
pomoshnik_pdn303,120
pomoshnik_pdn307,120
pomoshnik_pdn309,120
...,...
pomoshnik_pdn242,20
pomoshnik_pdn240,18
pomoshnik_pdn173,16


In [242]:
initial_task_for_today_absent_assistants =  \
initial_task_for_today[initial_task_for_today.operator_login.isin(absent_assistants.login)] \
[['emiasid', 'status', 'old_comment', 'mo', 'priority', 'month']]

In [243]:
additional_task_for_present_assistants = \
pd.concat([
#     new_doctor_patients,
    initial_task_for_today_absent_assistants
]).reset_index(drop=True)

In [244]:
additional_task_for_present_assistants_by_mo = \
additional_task_for_present_assistants.groupby(['mo']).agg({'emiasid': 'count'}).sort_values(by=['mo', 'emiasid'])

In [245]:
distribution_count = initial_task_for_today_present_assistants.groupby(['mo']) \
.agg({'emiasid': 'count', 'operator_login': 'nunique'}) \
.sort_values(by=['mo', 'emiasid'])

In [246]:
distribution_count['total_delta'] = distribution_count['operator_login'] * task_amount - distribution_count['emiasid']

In [247]:
distribution_count = distribution_count[distribution_count.total_delta > 0]

In [248]:
additional_task_for_present_assistants

,emiasid,status,old_comment,mo,priority,month
0,10823061,Будет в Москве в другом месяце,NaN,ДЦ 5,3,august
1,19053749,Будет в Москве в другом месяце,"не в Москве,сентябрь",ДЦ 5,3,august
2,30000000643031,Запишется самостоятельно,NaN,ДЦ 5,3,august
3,19335604,Будет в Москве в другом месяце,не в Москве.будет в сентябре.,ДЦ 5,3,august
4,17686600,Будет в Москве в другом месяце,"не в Москве , сентябрь",ДЦ 5,3,august
...,...,...,...,...,...,...
4039,21634808,Не дозвонились,NaN,ГП 22,5,september
4040,16042160,Не дозвонились,NaN,ГП 22,5,september
4041,10919276,Не дозвонились,NaN,ГП 22,5,september
4042,19036014,Не дозвонились,NaN,ГП 22,5,september


In [249]:
final_task_predistribution = \
pd.concat([initial_task_for_today_present_assistants, additional_task_for_present_assistants]).reset_index(drop=True)

In [250]:
empty_login_patients_amount_before = final_task_predistribution[final_task_predistribution.operator_login.isna()].shape[0]
empty_login_patients_amount_before

4044

In [251]:
%%time

for mo in distribution_count.index:
    
    while final_task_predistribution[final_task_predistribution.mo == mo].groupby('operator_login') \
    .agg({'emiasid': 'count'}).min()['emiasid'] < task_amount and \
    final_task_predistribution[(final_task_predistribution.operator_login.isna()) & (final_task_predistribution.mo == mo)] \
    .shape[0] > 0:
        
        miminal_in_mo_assistant_emiasid_amount = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('operator_login').agg({'emiasid': 'count'}).min()['emiasid']

        minimal_in_mo_emiasid_amount_login = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('operator_login').agg({'emiasid': 'count'}).sort_values(by='emiasid').index[0]
        
        
        empty_login_in_mo_index = final_task_predistribution[
            (final_task_predistribution.operator_login.isna()) & (final_task_predistribution.mo == mo)
        ].index[0]
        
        final_task_predistribution = \
        final_task_predistribution.sort_values(by=['priority', 'last_call_date', 'calls_amount'])

        final_task_predistribution.loc[empty_login_in_mo_index, 'operator_login'] = minimal_in_mo_emiasid_amount_login


CPU times: total: 10.5 s
Wall time: 10.7 s


In [252]:
empty_login_patients_amount_after = final_task_predistribution[final_task_predistribution.operator_login.isna()].shape[0]
empty_login_patients_amount_after

3562

In [253]:
final_task_distribution = final_task_predistribution[~final_task_predistribution.operator_login.isna()] \
.copy().reset_index(drop=True)

In [254]:
final_task_distribution.status.value_counts()

Не дозвонились                    8862
Перенос звонка на другой день     4744
Ни разу не звонили                2365
Разговор состоялся                1168
Перезвонить сегодня                371
Запишется самостоятельно           356
Будет в Москве в другом месяце     333
Напомнить                          229
Не нравится врач                    15
Не нравится МО                       8
Отказ                                6
Госпитализирован                     5
Живет не в Москве                    3
Пациент отрицает диагноз             1
Name: status, dtype: int64

In [255]:
final_task_distribution.operator_login.value_counts()

pomoshnik_pdn168    120
pomoshnik_pdn43     120
pomoshnik_pdn212    120
pomoshnik_pdn2      120
pomoshnik_pdn177    120
                   ... 
pomoshnik_pdn270     33
pomoshnik_pdn265     25
pomoshnik_pdn173     16
pomoshnik_pdn131      3
pomoshnik_pdn151      1
Name: operator_login, Length: 165, dtype: int64

In [256]:
final_task_distribution = final_task_distribution.rename(columns={'login': 'operator_login'})

In [257]:
final_task_distribution = final_task_distribution.rename(columns={'last_comment': 'old_comment'})

In [258]:
final_task_distribution.shape[0]

18466

In [271]:
final_task_distribution.groupby('operator_login').agg({'emiasid': 'count'}).sort_values(by=['emiasid']).head(30).sum()

emiasid    2266
dtype: int64

In [272]:
final_task_distribution['emiasid'] = final_task_distribution['emiasid']

In [273]:
final_task_distribution = final_task_distribution.drop_duplicates(subset=['emiasid'], keep='first')
final_task_distribution.shape[0]

18466

In [274]:
final_task_distribution.status.value_counts()

Не дозвонились                    8862
Перенос звонка на другой день     4744
Ни разу не звонили                2365
Разговор состоялся                1168
Перезвонить сегодня                371
Запишется самостоятельно           356
Будет в Москве в другом месяце     333
Напомнить                          229
Не нравится врач                    15
Не нравится МО                       8
Отказ                                6
Госпитализирован                     5
Живет не в Москве                    3
Пациент отрицает диагноз             1
Name: status, dtype: int64

In [275]:
final_task_distribution.last_call_date.value_counts()

2022-09-07    4718
2022-09-08    1876
2022-09-02    1796
2022-09-06    1760
2022-09-05    1301
2022-09-12     916
2022-09-09     804
2022-09-15     645
2022-09-14     341
2022-09-13     327
2022-08-16      28
2022-09-01      25
2022-08-17      20
2022-08-31      14
2022-08-15      14
2022-08-09      13
2022-08-23      11
2022-08-04      10
2022-08-03       9
2022-08-24       9
2022-08-10       8
2022-08-05       8
2022-08-12       7
2022-08-11       6
2022-08-18       6
2022-08-26       4
2022-08-08       3
2022-08-19       3
2022-08-22       3
2022-08-25       3
2022-08-29       2
2022-08-30       1
Name: last_call_date, dtype: int64

In [276]:
final_task_distribution[final_task_distribution.month == 'august']\
.to_excel(rf'august_task_for_today_{today_y_m_d}.xlsx', index=False)

In [277]:
final_task_distribution[final_task_distribution.month == 'september']\
.to_excel(rf'september_task_for_today_{today_y_m_d}.xlsx', index=False)

In [278]:
final_task_distribution.to_excel(rf'full_task_for_today_{today_y_m_d}.xlsx', index=False)

In [ ]:
tomorrow_record_patients[tomorrow_record_patients.emiasid.isin(final_task_distribution.emiasid)]

In [ ]:
final_task_distribution.